<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>CS 144 - Winter 2024 - Homework 7</h1></center>
<center><h1>Due: Sunday, March 10, 2024 @ 11:59pm</h1></center>

## Read *all* cells carefully and answer all parts (both text and code)

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter): Rayyan Zaid  <br>
    Your student ID (submitter): 862291205
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this homework, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline. 
<br>
<br>
<B>Late work:</B> Each student is granted five "late days" which can be used (in integer units) on any of the homework. If a more dire situation arises, please contact the instructor. DO NOT RESORT TO CHEATING.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Chapter 6: Motif finding</h1></center>
<br>

As usual, Chapter 6 has a many coding challenges. In the interest of time, we will <b>skip</b> the following coding challenges in the book
<UL>
<LI><A HREF="https://cogniterra.org/lesson/33948/step/8?unit=25794">Implement MotifEnumeration</A></LI>
<LI><A HREF="https://cogniterra.org/lesson/33950/step/9?unit=25796">Implement MedianString</A></LI>
<LI><A HREF="https://cogniterra.org/lesson/33951/step/5?unit=25797">Implement GreedyMotifSearch</A></LI>
<LI><A HREF="https://cogniterra.org/lesson/33952/step/9?unit=25798">Implement GreedyMotifSearch with pseudocounts</A></LI>
<LI><A HREF="https://cogniterra.org/lesson/33953/step/5?unit=25799">Implement RandomizedMotifSearch</A>
<LI><A HREF="https://cogniterra.org/lesson/33958/step/1?unit=25804">Implement DistanceBetweenPatternAndStrings</A>
</UL>
Those challenges above are OPTIONAL. They are good problems, but you do not need to solve it, unless you feel inclined to do so.

<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Computing the probability according to a PWM</h1></center>
<br>
In this homework, we will find motifs using position-weight matrix (or PWM, or profile) as a probabilistic model. A key component in the motif finding is to find the k-mer in a string that scores the highest probability according to a given PWM.

<div style="color: #000000;background-color: #EEFFEE">
    <font size=+2>Question 1</font> <font size=+1>(10 points)</font>

Solve the <A HREF="https://cogniterra.org/lesson/33951/step/3?unit=25797">Profile-most Probable k-mer Problem.</A> Submit your solution to the challenge on Cogniterra and provide your Python code below.

We are providing the code to parse the input text as generated by Cogniterra.

The wget below fetches the test file formatted as provided by Cogniterra.
</div>

In [2]:
!wget http://www.cs.ucr.edu/~stelo/cs144winter24/data/hw7_test1.txt

--2024-02-28 18:01:35--  http://www.cs.ucr.edu/~stelo/cs144winter24/data/hw7_test1.txt
Resolving www.cs.ucr.edu (www.cs.ucr.edu)... 169.235.30.15
Connecting to www.cs.ucr.edu (www.cs.ucr.edu)|169.235.30.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134 [text/plain]
Saving to: 'hw7_test1.txt'

     0K                                                       100% 1.39M=0s

2024-02-28 18:01:35 (1.39 MB/s) - 'hw7_test1.txt' saved [134/134]



In [6]:
# no libraries allowed, except for biopython and numpy, if wanted

def score_profile(s, p):
    """ computes the probability of string s using the profile p """
    
    totalScore = 1
    for i in range(len(s)):
        letter = s[i]
        prob = p[i][letter]
        totalScore*=prob

    return totalScore

# read the file
f = open("dataset_33951_3.txt", "r")
dataset = ''
for x in f:
    dataset += x
# parse the input
dataset = dataset.splitlines()                                          
text = dataset[0]                                                       
k = int(dataset[1])                                                     
As = list(map(float, dataset[2].split())) # probabilities for As
Cs = list(map(float, dataset[3].split())) # probabilities for Cs
Gs = list(map(float, dataset[4].split())) # probabilities for Gs
Ts = list(map(float, dataset[5].split())) # probabilities for Ts
# build the PWM p as an array of dictionaries
p = []                                                                  


for i in range(len(As)):                                               
    p.append({'A': As[i], 'C': Cs[i], 'G': Gs[i], 'T': Ts[i]})
# keep track of the best scoring position in the variable best
# best[0] contains the score, and best[1] the best k-mer
best = (-1, '')                                                         
for i in range(len(text)-k+1):                                        
    score = score_profile(text[i:i+k], p)                               
    if score > best[0]:                                                 
        best = (score, text[i:i+k])
print(best[1])

TGAAATACGCTTAGG


<hr style="height:5px;border-width:2;color:gray;background-color:#000000"> 
<center><h1>Gibbs sampler</h1></center> 
<br>

We will be using `score_profile` in our Gibbs sampler.

<div style="color: #000000;background-color: #EEFFEE">
    <font size=+2>Question 2:</font> <font size=+1>(20 points)</font>

Implement <A HREF="https://cogniterra.org/lesson/33955/step/4?unit=25801">GibbsSampler</A> as described in the book/slides. Submit your solution to the challenge on Cogniterra and provide your Python code below. 

We are providing the code to parse the input text as generated by Cogniterra. We also providing the code `profile_pseudo`, `count`, and the skeleton of `GibbsSampler`. Please complete the code for `GibbsSampler`. As usual, you are free to use this code, or start from scratch.

The wget below fetches the test file formatted as provided by Cogniterra.
</div>

In [1]:
!wget http://www.cs.ucr.edu/~stelo/cs144winter24/data/hw7_test2.txt

--2024-02-28 18:01:24--  http://www.cs.ucr.edu/~stelo/cs144winter24/data/hw7_test2.txt
Resolving www.cs.ucr.edu (www.cs.ucr.edu)... 169.235.30.15
Connecting to www.cs.ucr.edu (www.cs.ucr.edu)|169.235.30.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173 [text/plain]
Saving to: 'hw7_test2.txt'

     0K                                                       100% 3.19M=0s

2024-02-28 18:01:25 (3.19 MB/s) - 'hw7_test2.txt' saved [173/173]



In [9]:
import random

def profile_pseudo(S):
    """ Given a vector of strings S of equal length computes the matrix profile of S, i.e.,
        the probability of As, Cs, Gs and Ts in each column, including a pseudo-count of 1; each
        column is scaled by the number of strings + 4, so that the probabilities in each column sum up to one"""
    p = []
    n = len(S[0])
    # we are adding one additional occurrence (pseudo-count)
    # of A, C, G and T to the columns, so the total # of symbols
    # column-wise is t = len(S) + 4
    t = len(S) + 4
    # compute probability distribution column by column
    for i in range(n):
        p.append({'A': 1.0/t, 'T': 1.0/t, 'C': 1.0/t, 'G': 1.0/t}) # pseudo-count of 1
        for s in S:
            p[i][s[i]] += (1.0 / t)
    return p

def count(S):
    """ Given a vector S of strings of equal length computes the count profile of S, i.e.,
        counts of As, Cs, Gs and Ts in each column """
    p = []
    n = len(S[0])
    # compute counts column by column
    for i in range(n):
        p.append({'A': 0.0, 'T': 0.0, 'C': 0.0, 'G': 0.0})
        for s in S:
            p[i][s[i]] += 1.0
    return p

import random

def GibbsSampler(S, k, t, N):
    n = len(S[0])
    best = (0, [])
    m = []

    for i in range(t):
        j = random.randrange(n-k+1)
        m.append(S[i][j:j+k])

    for _ in range(N):
        # 1. Score the k-mers in m
        motifs_count = count(m)
        score = 0
        for col in motifs_count:
            max_count = max(col.values())
            score += max_count
        
        # 2. Remember the best set of k-mers
        if score > best[0]:
            best = (score, list(m))
        
        # 3. Select a string i at random to be ignored
        i = random.randrange(t)

        # 4. Build the profile excluding the i-th string
        excluded_motifs = [m[j] for j in range(t) if j != i]
        p = profile_pseudo(excluded_motifs)

        # 5. Compute the distribution of probabilities
        d = []
        for j in range(n-k+1):
            kmer = S[i][j:j+k]
            d.append(score_profile(kmer, p))
        
        # 6. Generate a random position according to the distribution d
        sum_of_distr = sum(d)
        if sum_of_distr == 0:
            randpos = random.randrange(n-k+1)
        else:
            d = [x/sum_of_distr for x in d]
            r = random.random()
            f = 0
            for x in range(len(d)):
                f += d[x]
                if f >= r:
                    randpos = x
                    break
                    
        # 7. Update the k-mer for the i-th string
        m[i] = S[i][randpos:randpos+k]
    return best


## MAIN
f = open("dataset_33955_4.txt", "r")
dataset = ''
for x in f:
    dataset += x
# print(dataset)

# parse the input
dataset = dataset.strip().split('\n')
k, t, N = list(map(int, (dataset[0].split(' '))))
S = dataset[1:]
assert(len(S)==t)

# we remember the optimal motif in best
best = (0, [])

# run Gibbs sampler with 20 random re-starts
for _ in range(20):
    r = GibbsSampler(S, k, t, N)
    if r[0] > best[0]:
        best = r
print('\n'.join(best[1]))

TCGCGTCTATGGGAT
TCGACCCTAAGGACC
TCGGGACTAAACGCC
TGTAGACTAAGGACC
TCGGGACTAAGGGAT
GAAGGACTAAGGACC
TCGGGACTAGCTACC
TCGCAGCTAAGGACC
TCGGGATATAGGACC
TCGGGACTAAGTGAC
TCGGGAACTAGGACC
TCGGGACCTGGGACC
TCGGGTGAAAGGACC
TCGGTGTTAAGGACC
TCGGGACTTCTGACC
CCGGGACTAAGGAAT
CAGGGACTAAGGACA
TCGGTCATAAGGACC
TCCCTACTAAGGACC
TCGGGGAAAAGGACC
